# The Sopranos
### Рейтинг серий и режиссеры

#### Парсинг и предобработка данных для визуализации информации о рейтингах и режиссерах серий сериала "Сопрано" в Power BI

Загрузим необходимые библиотеки

In [1]:
import pandas as pd  # Для работы с данными в pd.DataFrame

import seaborn as sns  # Для визуализации данных

import time  # Для работы с временными задержками

import requests  # Для HTTP-запросов
from bs4 import BeautifulSoup  # Для парсинга HTML

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

import time  # Для пауз между действиями

%matplotlib inline

Парсим данные о режиссерах при помощи библиотек Selenium и BeautifulSoup. \
Note: классы элементов на сайте IMDb могут меняться. Если скрипт перестал работать, надо проверить актуальные классы через DevTools

Создадим функцию для парсинга

In [34]:
def parse_imdb_directors_episodes(driver, url, save_path=None):
    '''
    Парсит информацию о режиссерах и эпизодах сериала "The Sopranos" с IMDb
    
    Параметры:
        driver: Selenium WebDriver
        url (str): URL страницы для парсинга
        save_path (str/None): путь для сохранения CSV (если None - не сохранять)
    
    Возвращает:
        pd.DataFrame: датафрейм с полями ['director', 'season_ep']
    '''
    
    # Список для хранения данных о режиссерах
    directors = []

    try:
        driver.get(url)  # Открываем страницу в браузере
            
        # Ожидаем появления якоря на секцию с режиссерами
        directors_anchor = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//a[@href='#director']"))
        )
        
        # Находим секцию с режиссерами
        directors_section = directors_anchor.find_element(By.XPATH, './ancestor::section[1]')
        
        # Находим всех режиссеров в секции
        directors_list = directors_section.find_elements(By.XPATH, ".//li[contains(@class, 'ipc-metadata-list-summary-item')]")
        
        # Обрабатываем каждого режиссера
        for director in directors_list:
            try:
                # Ищем кнопку "episodes"
                episodes_buttons = director.find_elements(By.XPATH, ".//button[contains(@class, 'ipc-link') and contains(., 'episode')]")
                    
                # Кликаем на кнопку
                ActionChains(driver).move_to_element(episodes_buttons[0]).click().perform()
                
                # Ожидаем появления попапа
                popup = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "div.ipc-popover__content, div[role='dialog']"))
                )
                
                # Ждем загрузки контента в попапе
                time.sleep(1)
                
                # Парсим HTML попапа
                popup_html = popup.get_attribute('outerHTML')
                soup = BeautifulSoup(popup_html, 'lxml')
                
                # Извлекаем имя режиссера
                dir_name_elem = soup.find('h3', class_='ipc-title__text')
                dir_name = dir_name_elem.text.strip()
                
                # Пробуем найти вкладки с сезонами
                season_tabs = driver.find_elements(By.CSS_SELECTOR, "li.ipc-tab[role='tab']")
                
                if season_tabs:
                    # Обрабатываем каждый сезон
                    for tab in season_tabs:
                        try:
                            # Кликаем на вкладку сезона
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
                            ActionChains(driver).move_to_element(tab).click().perform()
                            
                            time.sleep(1)  # Пауза для загрузки
                            
                            # Обновляем HTML после переключения вкладки
                            popup_html = popup.get_attribute('outerHTML')
                            soup = BeautifulSoup(popup_html, 'lxml')
                            
                            # Находим все эпизоды, снятые режиссером
                            episodes = soup.find_all('a', class_=lambda x: x and 'episodic-credits-bottomsheet__menu-item' in x)
                            
                            for episode in episodes:
                                season_ep = episode.find('li', class_='ipc-inline-list__item')
                                directors.append({
                                    'director': dir_name,
                                    'season_ep': season_ep.text.strip()
                                })
                        except Exception as e:
                            print(f"Ошибка при обработке вкладки сезона: {str(e)}")
                            continue
                else:
                    # Обработка случая без вкладок (один эпизод)
                    episode = soup.find('a', class_=lambda x: x and 'episodic-credits-bottomsheet__menu-item' in x)
                    season_ep = episode.find('li', class_='ipc-inline-list__item')
                    directors.append({
                        'director': dir_name,
                        'season_ep': season_ep.text.strip()
                    })
                    
            except Exception as e:
                print(f"Ошибка при обработке режиссера: {str(e)}")
                
            finally:
                # Закрываем попап
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                time.sleep(0.5)
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")
    finally:
        # Создаем pd.DataFrame
        df = pd.DataFrame(directors)
        
        if save_path:
            df.to_csv(save_path, index=False)
            print(f"Данные сохранены в {save_path}")
        
        return df

Применим функцию

In [35]:
if __name__ == "__main__":
    
    # Настройки Chrome для стабильной работы
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')  # Открываем браузер на полный экран
    options.add_experimental_option('detach', True)  # Браузер не закроется автоматически после скрипта

    # Инициализация драйвера Chrome с автоматической установкой нужной версии
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    # URL для парсинга
    url = 'https://www.imdb.com/title/tt0141842/fullcredits'
    
    # Запуск парсера
    sopranos_directors_df = parse_imdb_directors_episodes(
        driver=driver,
        url=url,
        save_path='data/raw/sopranos_directors.csv'
    )
    
    print(f"Успешно собрано {len(sopranos_directors_df)} записей")
    driver.quit()

Данные сохранены в sopranos_directors.csv
Успешно собрано 86 записей


Посмотрим на итоговый датафрейм

In [36]:
sopranos_directors_df

,director,season_ep
0,Timothy Van Patten,S1.E8
1,Timothy Van Patten,S2.E4
2,Timothy Van Patten,S2.E5
3,Timothy Van Patten,S2.E11
4,Timothy Van Patten,S3.E2
...,...,...
81,Danny Leiner,S6.E7
82,David Nutter,S6.E2
83,Steve Shill,S6.E10
84,Phil Abraham,S6.E15


Далее спарсим рейтинги серий. \
Создадим функцию для парсинга

In [2]:
def parse_imdb_sopranos_ratings(base_url, save_path=None):
    '''
    Парсит рейтинги эпизодов сериала "The Sopranos" с IMDB
    
    Параметры:
        base_url (str): URL страницы для парсинга
        save_path (str/None): путь для сохранения CSV (если None - не сохранять)
    
    Возвращает:
        pd.DataFrame: датафрейм с полями ['season', 'ep_num', 'ep_title', 'ep_rating']
    '''
    
    seasons = []  # Список для хранения данных
    
    # Проходим по всем сезонам (от 1 до 6)
    for season in range(1, 7):
        
        # Формируем URL для конкретного сезона
        url = f"{base_url}?season={season}"
        
        # Заголовки для имитации браузера и избежания блокировки
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9"
        }
        
        # Отправляем GET-запрос и получаем ответ
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Находим все контейнеры с эпизодами
        episode_containers = soup.find_all('article', class_='sc-fcf4d924-1 emwOvP episode-item-wrapper')
        
        # Обрабатываем каждый эпизод
        for episode in episode_containers:
            
            # Извлекаем заголовок эпизода
            title = episode.find('div', class_='ipc-title__text').text.strip()
            
            # Извлекаем номер эпизода (формат "S01.E01 Название")
            episode_number = title.split('.E')[1].split(' ')[0]
            
            # Извлекаем название эпизода
            ep_title = title.split(' ∙ ')[1]
            
            # Извлекаем рейтинг эпизода
            rating = episode.find('span', class_='ipc-rating-star--rating').text.strip()
            
            # Добавляем данные в список
            seasons.append({
                'season': season,
                'ep_num': episode_number,
                'ep_title': ep_title,
                'ep_rating': rating
            })

    # Создаем pd.DataFrame
    df = pd.DataFrame(seasons)

    if save_path:
        df.to_csv(save_path, index=False)
        print(f"Данные сохранены в {save_path}")

    return df

Применим функцию

In [4]:
if __name__ == "__main__":

    # URL для парсинга
    base_url = 'https://www.imdb.com/title/tt0141842/episodes'
    
    # Запуск парсера
    sopranos_ratings_df = parse_imdb_sopranos_ratings(
        base_url=base_url,
        save_path='data/raw/sopranos_ratings.csv'
    )
    
    print(f"Успешно собрано {len(sopranos_ratings_df)} записей")

Данные сохранены в data/raw/sopranos_ratings.csv
Успешно собрано 86 записей


Посмотрим на итоговый датафрейм

In [46]:
sopranos_ratings_df

,season,ep_num,ep_title,ep_rating
0,1,1,Pilot,8.4
1,1,2,46 Long,8.2
2,1,3,"Denial, Anger, Acceptance",8.5
3,1,4,Meadowlands,8.5
4,1,5,College,8.9
...,...,...,...,...
81,6,17,Walk Like a Man,8.9
82,6,18,Kennedy and Heidi,9.2
83,6,19,The Second Coming,9.3
84,6,20,The Blue Comet,9.6


### Предобработка данных

Теперь займемся предобработкой данных

In [49]:
# Загружаем данные из CSV-файлов

sprns_rat_df = pd.read_csv('data/raw/sopranos_ratings.csv')

In [50]:
sprns_dir_df = pd.read_csv('data/raw/sopranos_directors.csv')

Сначала поработаем с датафреймом с рейтингами

In [68]:
# Посмотрим на структуру датафрейма

sprns_rat_df.head()

,season,ep_num,ep_title,ep_rating,season_ep_num
0,1,1,Pilot,8.4,1.1
1,1,2,46 Long,8.2,1.2
2,1,3,"Denial, Anger, Acceptance",8.5,1.3
3,1,4,Meadowlands,8.5,1.4
4,1,5,College,8.9,1.5


In [65]:
# Создадим комбинированное поле "сезон.эпизод"

sprns_rat_df['season_ep_num'] = sprns_rat_df.season.astype(str) + '.' + sprns_rat_df.ep_num.astype(str)

Добавим поле с рейтинговыми группами

In [77]:
bins = [0, 8.0, 8.5, 9.0, 9.5, 10]
labels = ['Below 8.0', '8.0–8.5', '8.5–9.0', '9.0–9.5', '9.5 or above']
sprns_rat_df['rating_range'] = pd.cut(sprns_rat_df.ep_rating, 
                                      bins=bins, 
                                      labels=labels, 
                                      right=False)  # Интервал включает левую границу, но не правую (например, 8.0 попадает в "8.0–8.5")

Посмотрим на итоговый датафрейм

In [84]:
sprns_rat_df.head()

,season,ep_num,ep_title,ep_rating,season_ep_num,rating_range
0,1,1,Pilot,8.4,1.1,8.0–8.5
1,1,2,46 Long,8.2,1.2,8.0–8.5
2,1,3,"Denial, Anger, Acceptance",8.5,1.3,8.5–9.0
3,1,4,Meadowlands,8.5,1.4,8.5–9.0
4,1,5,College,8.9,1.5,8.5–9.0


Сохраняем обработанные данные о рейтингах

In [85]:
# sprns_rat_df.to_csv('data/processed/sopranos_ratings_edited.csv', index=False)

Теперь поработаем с датафреймом с режиссерами

In [53]:
# Извлечем номер сезона из записи формата "S01E01"

sprns_dir_df['season'] = sprns_dir_df.season_ep.str.split('.').str[0].str[1]

In [54]:
# Извлечем номер эпизода

sprns_dir_df['ep_num'] = (sprns_dir_df.season_ep.str.split('.').str[1].str[1:]).astype('int')

In [55]:
# Создадим комбинированное поле "сезон.эпизод"

sprns_dir_df['season_ep_num'] = sprns_dir_df.season + '.' + sprns_dir_df.ep_num.astype(str)

In [56]:
# Удалим исходное поле 

sprns_dir_df.drop(columns=['season_ep'], inplace=True)

In [60]:
# Просмотр информации о DataFrame с режиссерами

sprns_dir_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   director       86 non-null     object
 1   season         86 non-null     object
 2   ep_num         86 non-null     int32 
 3   season_ep_num  86 non-null     object
dtypes: int32(1), object(3)
memory usage: 2.5+ KB


Сохраняем обработанные данные о режиссерах

In [40]:
# sprns_dir_df.to_csv('data/processed/sopranos_directors_edited.csv', index=False)